In [10]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote, urlparse
import re
import time

URL = "https://display.wconcept.co.kr/rn/women"

service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)
time.sleep(2)

driver.find_element(By.XPATH, "//span[text()='베스트']").click()
time.sleep(5)

# ✅ 상품 카드 가져오기
items = driver.find_elements(By.CSS_SELECTOR, "li")

print("상품 수:", len(items))

for item in items[:20]:   # 처음 20개만 테스트
    try:
        brand = item.find_element(By.CSS_SELECTOR, "strong").text
    except:
        brand = ""

    try:
        name = item.find_element(By.CSS_SELECTOR, "p").text
    except:
        name = ""

    try:
        sale = item.find_element(By.CSS_SELECTOR, "em").text
    except:
        sale = ""

    print(brand, name, sale)

driver.quit()

상품 수: 0


InvalidSessionIdException: Message: invalid session id; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#invalidsessionidexception
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0xd37dd3
	0xd37e14
	0xb41bee
	0xb7f2d5
	0xbae8f6
	0xbaa491
	0xba9ab2
	0xb1500e
	0xb155ae
	0xb15a8d
	0xf96924
	0xf91bf7
	0xfaf5a0
	0xd50f58
	0xd5891d
	0xb14c69
	0xb142b0
	0x10f876f
	0x764ffcc9
	0x773282ae
	0x7732827e


In [ ]:

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from urllib.parse import quote, urlparse
import re
import time
import pickle

URL = "https://www.instagram.com/"

service = Service(ChromeDriverManager().install())
options = Options()

# options.add_argument("--headless")
# options.add_argument("--disable-gpu")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--window-size=1920,1080")
options.add_argument("--start-maximized")
options.add_argument("--user-agent=Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/144.0.0.0 Safari/537.36")
options.add_argument("--lang=ko_KR")
options.add_argument("--no-sandbox")

driver = webdriver.Chrome(service=service, options=options)
driver.get(URL)

# Refactoring을 위한 유틸리티 함수
def wait_post_loaded(driver, timeout) :
    wait = WebDriverWait(driver, timeout)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "main")))
    wait.until(lambda d: len(d.find_elements(By.XPATH, "//main//a[@href]")) > 0)

def find_scroll_container_in_main(driver) :
    main = driver.find_element(By.TAG_NAME, "main")
    canditates = main.find_elements(By.XPATH, ".//*[contains(@class, 'x78zum5') or contains(@style, 'overflow')]")

    best = None
    best_h = 0

    for el in canditates :
        try :
            h = driver.execute_script("return arguments[0].clientHeight;", el)
            sh = driver.execute_script("return arguments[0].scrollHeight;", el)

            if sh > h and h > best_h :
                best = el
                best_h = h
        except :
            pass

    return best

def scroll_inside(driver, el, max_round, pause) :
    last = -1
    for _ in range(max_round) :
        driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight;", el)
        time.sleep(pause)
        cur = driver.execute_script("return arguments[0].scrollTop;", el)
        if cur == last :
            break
        last = cur

def extract_username_from_href(href) :
    if not href:
        return None

    path = urlparse(href).path.strip("/")

    if not path :
        return None

    if path.startswith(("p/", "reel/", "tv/", "explore/", "accounts/")) :
        return None

    username = path.split("/")[0]

    if re.fullmatch(r"[A-Za-z0-9._]{1,30}", username) :
        return username

    return None

    # url = "https://www.instagram.com/musinsa.official/path=/utm=""/"
    # parsed = urlparse(url)
    # parsed = {"scheme":"https", "netloc":"www.instagram.com", "path":"/musinsa.official/"}

def get_author_id(driver) :
    main = driver.find_element(By.TAG_NAME, "main")
    anchors = main.find_elements(By.XPATH, ".//a[@href]")

    best = None
    best_y = 10**9

    for a in anchors :
        href = a.get_attribute("href") or ""
        username = extract_username_from_href(href)

        if not username :
            continue

        txt = (a.text or "").strip()
        if txt.startswith("@") :
            continue

        try:
            y = a.location["y"]
            if y < best_y :
                best_y = y
                best = username
        except :
            pass
            
    return best 

def get_comments(driver) :
    main = driver.find_element(By.TAG_NAME, "main")
    spans = main.find_elements(By.XPATH, ".//span[@dir='auto' and not(ancestor::*[@role='button'])]")

    comments = []
    
    for s in spans :
        t = s.text.strip()

        if not t :
            continue

        if re.fullmatch(r"\s*\d+\s*(주|일|시간|분|초)", t) :
            continue

        if len(t) <= 1 :
            continue

        comments.append(t)

    seen = set()
    uniq = list()
    
    for c in comments :
        if c not in seen :
            seen.add(c)
            uniq.append(c)
    return uniq

        # 정규표현식 : 특정한 형태의 패턴을 가지고 있는 문자열을 찾아내고 싶을 때 사용!
        # (주|일|시간|분|초) : | => vertical bar => or
        # \s* : * => 0개 또는 그 이상 => 아무것도 존재하지 않을수도 있고, 복수의 값이 있을 수도 있다
        # \s : 여백, 공백 등의 white space
        # 10 주 10주
        # \d+ : + => 1개 이상이 존재, \d => decimal => 정수
        # 10주
        # 엔터     엔터
        # 똑같은 댓글 -> 같은댓글
    
with open("instagram_cookies.pkl", "rb") as f :
    cookies = pickle.load(f)

for cookie in cookies :
    driver.add_cookie(cookie)

driver.refresh()
time.sleep(10)

try :
    btn = driver.find_element(By.XPATH, "//button[normalize-space()='나중에 하기' or normalize-space()='정보 저장']")
    btn.click()
    time.sleep(2)
except :
    pass

print("정상적인 로그인 후 나중에 하기 버튼 클릭 완료")

base_url = "https://www.instagram.com"
results = list()

brands = (
    df_top1["브랜드명"]
    .dropna()
    .astype(str)
    .str.strip()
    .unique()
    .tolist()
)

print("브랜드 수: ", len(brands))

for keyword in brands :
    encoded = quote(keyword)
    tag_url = f"{base_url}/explore/tags/{encoded}"
    driver.get(tag_url)
    time.sleep(5)

    post_link = set()

    for _ in range(2) :
        links = driver.find_elements(By.TAG_NAME, "a")
        for link in links :
            href = link.get_attribute("href")
            if href and "/p/" in href :
                if href.startswith("/p/") :
                    href = base_url + href
                post_link.add(href)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)

    print(f"[{keyword}] 수집된 게시물 수 :", len(post_link))

    for url in list(post_link) :
        driver.get(url)

        try :
            wait_post_loaded(driver, timeout=15)
            
        except Exception as e :
            print(f"페이지 로딩 실패:, {keyword} | {url} | {e}")
            results.append({"brand": keyword, "url": url, "author_id": None, "comments": []})
            continue

        try :
            scroll_box = find_scroll_container_in_main(driver)
            if scroll_box :
                scroll_inside(driver, scroll_box, max_round=10, pause=1)
        except :
            pass

        try :
            author_id = get_author_id(driver)
            comments = get_comments(driver)
        except Exception as e :
            print(f"추출 실패: {keyword} | {url} | {e}")
            author_id, comments = None, []

        results.append({
            "brand": keyword,
            "url": url,
            "author_id": author_id,
            "comments": comments
        })

for r in results[:20] :
    print(r["brand"], r["url"])
    print("author:", r["author_id"])
    print("comments:", r["comments"])
    print("=" * 60)
    
driver.quit()

print("총 결과 수: ", len(results))